# Cleaning Dataset

In [40]:
import pandas as pd
import numpy as np
import csv
from datetime import timedelta
from dateutil import parser

global powS
global powG
global time 
global state
global down 
global c
global c1
global Gsay
global Esay
global c_esp
global c_w 

global p_s
global g_s
global t_c


In [41]:
def finish():
    

    global p_s 
    p_s = []
    p_s.append(0)
    p_s.append(0)
    global g_s
    g_s = []
    g_s.append(0)
    g_s.append(0)
    global Gsay
    Gsay =0
    global Esay 
    Esay =0
    global c_esp 
    c_esp = 0
    global c_w  
    c_w= 0
    global state 
    state = "000"
    global t_c
    t_c = 0


In [3]:
def filter1(day,month,year):
    date = str(day)+ "-"+str(month) + "-"+ str(year)
    ds = pd.read_csv("data/espresso"+date+".csv",sep=",")
    
    global t
    t=0
    global t_s
    t_s=0
    global powS 
    powS= ds['PowS']
    global powG 
    powG = ds['PowG']
    global time 
    time = ds['time']
    global state
    state = "000"
    global down 
    down = 0
    global c
    c =11
    global c1
    c1 =3
    global Gsay
    Gsay=0
    global Esay
    Esay =0
    global c_esp 
    c_esp =0
    global c_w
    c_w= 0

    ### 2 arrays to collect the data for only one case of an 
    global p_s
    p_s=[]
    global g_s
    g_s=[]
    
    global t_c
    t_c = 0
    
    resultFile = open("data/cleaned/data"+date+".csv",'w', newline='')
    wr = csv.writer(resultFile)

    # for starting and ending time
    g_s.append(0)
    g_s.append(0)
    p_s.append(0)
    p_s.append(0)

    valid = False
    

    for i,j,k in zip(powS,powG,time): 
        # i represents Power from grinder
        # j represents Power from Espresso
        # k represents time
        ######### checking the grinder

        if state!="000" and state!="101" and state!="100" and state!="110": ## grinder states
            # if its a grinder state we add it to the grinder list
            g_s.append(j)
            t_s+=1
        if state!="000" and state!="001" and state!="010" and state!="011": ## espresso states
            # if its an espresso state we add it to the espresso list
            p_s.append(i)
            t+=1
        if j>100 and state == "000": 
            state="001"
            k = parser.parse(k)
            g_s[0]= k
        if (j<100) and state =="001":
            state = "010"
        if c!=0 and state == "010":
            if j>100:
                state = "011"
            else:
                c-=1
        if c==0 and state =="010":
            if j>100:
                state ="011"
            else:
                state ="100"
                c = 11
                Gsay = 1
        if c1>2 and state=="011":
            if j<100:
                Gsay = 1
                c= 11
                c1 = 3
                state ="100"
            else:
                c1-=1
        if (c1 >=0 or c1==0) and state =="011":
            if j<100:
                Gsay = 2    
                c= 11
                c1 = 3
                state ="100"
        ############# checking the espresso machine
        if state == "100":
            p_s[0] = k = parser.parse(k)
            if t_c>180:
                valid = False
                state="110"
            if i>40 and i<60:
                state = "101"
            else:
                t_c +=1
        if state == "101":
            if c_esp>60:
                state="110"
            if i>=50:
                c_w=0
                c_esp+=1
            elif i<50:
                if c_w<3:
                    c_w+=1
                    c_esp+=1
                if c_w>=3 and c_esp>30:
                    valid = True
                    state = "110"
        if state =="110":
            if valid == False:
                g_s[2] = -150
                p_s[2] = -150
            if c_esp >=45:
                Esay = 2
            elif c_esp<45:
                Esay = 1
            g_s[1]=  (g_s[0] + timedelta(seconds = t))
            t = 0
            p_s[1] = (p_s[0]+ timedelta(seconds = t_s))
            t_s=0
            while len(p_s)<200:
                p_s.append(0)
            while len(g_s)<200:
                g_s.append(0)
            if len(p_s) >200:
                p_s = p_s[:200]
                p_s[2] =  -150
                g_s[2] = -150
            if len(g_s) >200:
                g_s = g_s[:200]
                p_s[2] =  -150
                g_s[2] = -150
            # transposing the columns to rows for later to be feeded to a neural network or LSTM
            if p_s[2] !=-150 and g_s[2]!=-150:
                p = np.transpose(p_s)
                g = np.transpose(g_s)

                wr.writerows([g])
                wr.writerows([p])

            finish()